# 独立实现情感分类

> 1. 数据准备
2. 建立模型，编译
3. 训练模型
4. 进行评估

## 数据准备

等会将数据储存在csv中拷贝过来

## 建立模型，双编码长短记忆模型，只使用一个LSTM层

In [2]:
import keras.models
import pandas as pd
import numpy as np

In [7]:
res=np.load('imdb.npz')
#将训练数据和测试数据加载进来
x_train=res['x_train']
y_train=res['y_train']
x_test=res['x_test']
y_test=res['y_test']
#将数据分好
res.files

In [8]:
# 需要统计出最大的一个index，然后加1，作为嵌入层的参数
vocab_sizes=np.max([np.max(x) for x in x_train])

In [9]:
vocab_sizes=vocab_sizes+1
vocab_sizes

88585

> 在使用深度学习进行文本相关的分析的时候，需要记住文本预处理的相关步骤：
1. 文本常规的预处理，清洗文本
2. 然后每一个文档转化成一个向量，这里有两种方式：
    * 一种是直接使用word2vec对每个文档进行向量化
    * 另外一种是直接使用keras中的嵌入层实现文档向量化
    
    
    
在使用keras中的嵌入层进行词嵌入操作前需要进行如下几个操作：
1.文字拆分
2.建立索引
3.序列补齐（padding）
4.转换为矩阵
    


In [34]:
#进行序列补齐和转换为矩阵，
#这个训练文档已经是一个矩阵，但是该矩阵的长度不一样，所以需要将长度限定一下，全部限定在300个长度
from keras.preprocessing import sequence
x_train_pad=sequence.pad_sequences(sequences=x_train,maxlen=400)
x_test_pad=sequence.pad_sequences(sequences=x_test,maxlen=400)

In [35]:
x_train_pad.shape

(25000, 400)

In [36]:
#开始建立一个LSTM模型，首先需要确定使用的是序列模型，还是函数式模型
#这里准备使用序列模型

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from keras.optimizers import SGD 

In [37]:
model=Sequential()

In [14]:
#如何确定嵌入层的参数，
#嵌入层的功能是？参数有什么？，词典最大index，输入的个数，输出的维度。
#嵌入层的功能是：将所有的索引号映射到一个致密的低维向量中，

In [38]:
vocab_sizes

88585

In [64]:
model=Sequential()
model.add(Embedding(input_dim=vocab_sizes,output_dim=300,input_length=400))#添加一个嵌入层，对词进行向量化，

model.add(LSTM(200,return_sequences=True,dropout=0.2,kernel_initializer='random_normal'))#添加lstm层
model.add(LSTM(units=32,dropout=0.2))
model.add(Dense(activation='sigmoid',units=1))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 400, 300)          26575500  
_________________________________________________________________
lstm_25 (LSTM)               (None, 400, 200)          400800    
_________________________________________________________________
lstm_26 (LSTM)               (None, 32)                29824     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 27,006,157
Trainable params: 27,006,157
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
sgd=SGD(lr=0.01)

In [65]:
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics =['accuracy'])

In [66]:
model.fit(epochs=10,x=x_train_pad,y=y_train,batch_size=100,validation_data=(x_test_pad,y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 3143s - loss: 0.4781 - acc: 0.7812 - val_loss: 0.3381 - val_acc: 0.8610
Epoch 2/10
25000/25000 [==============================] - 3039s - loss: 0.2842 - acc: 0.8884 - val_loss: 0.3549 - val_acc: 0.8729
Epoch 3/10
25000/25000 [==============================] - 3006s - loss: 0.2067 - acc: 0.9258 - val_loss: 0.3675 - val_acc: 0.8713
Epoch 4/10
25000/25000 [==============================] - 2883s - loss: 0.1462 - acc: 0.9478 - val_loss: 0.4013 - val_acc: 0.8606
Epoch 5/10
25000/25000 [==============================] - 3008s - loss: 0.1077 - acc: 0.9632 - val_loss: 0.3971 - val_acc: 0.8668
Epoch 6/10
25000/25000 [==============================] - 2969s - loss: 0.0817 - acc: 0.9712 - val_loss: 0.6942 - val_acc: 0.8147
Epoch 7/10
25000/25000 [==============================] - 3028s - loss: 0.0646 - acc: 0.9784 - val_loss: 0.4626 - val_acc: 0.8585
Epoch 8/10
25000/25000 [================

In [63]:
scores = model.evaluate(x_test_pad, y_test)
print(scores)

25000/25000 [==============================] - 821s   
[0.6916507942461968, 0.84743999999999997]
